In [ ]:
!pip install streamlit pyngrok textblob vaderSentiment scikit-learn matplotlib pandas


In [ ]:
from pyngrok import ngrok
ngrok.kill()  # close old tunnels if any
ngrok.set_auth_token("34YNTAGVw3aYDYO61Rg8LIQjmf1_5N8ojgpMFqA3ayw73vEEE")


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import time
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt

st.set_page_config(page_title="Sentiment Dashboard", layout="wide")
st.title("🧠 Sentiment Dashboard — VADER & TextBlob")

st.markdown("""
Upload a CSV and select the text column.
Subpages: **VADER Sentiment**, **TextBlob Sentiment**, **Comparison Dashboard**.
Labels are derived using polarity:
- polarity > 0.05 → **Positive**
- polarity < -0.05 → **Negative**
- otherwise → **Neutral**
""")

# Sidebar: upload + navigation
st.sidebar.header("1) Upload & Navigation")
uploaded_file = st.sidebar.file_uploader("Upload CSV", type=["csv"])
page = st.sidebar.selectbox("Go to", ["VADER Sentiment", "TextBlob Sentiment", "Comparison Dashboard"])

if uploaded_file is None:
    st.info("Please upload a CSV file using the sidebar to begin.")
    st.stop()

# Read dataset
try:
    df = pd.read_csv(uploaded_file)
except Exception as e:
    st.error(f"Failed to read CSV: {e}")
    st.stop()

# Choose text column
text_col = st.sidebar.selectbox("Select text column", df.columns, index=0)

# helper
def polarity_to_label(score, pos_thresh=0.05, neg_thresh=-0.05):
    if score > pos_thresh:
        return "Positive"
    elif score < neg_thresh:
        return "Negative"
    else:
        return "Neutral"

# Initialize default times
vader_exec_time = 0.0
tb_exec_time = 0.0

# Compute VADER polarity & label
if "_vader_compound" not in df.columns or "_vader_label" not in df.columns:
    vader = SentimentIntensityAnalyzer()
    t0 = time.time()
    df["_vader_compound"] = df[text_col].astype(str).apply(lambda t: vader.polarity_scores(t)["compound"])
    df["_vader_label"] = df["_vader_compound"].apply(lambda s: polarity_to_label(s))
    vader_exec_time = time.time() - t0

# Compute TextBlob polarity & subjectivity & label
if "_tb_polarity" not in df.columns or "_tb_label" not in df.columns:
    t1 = time.time()
    df["_tb_polarity"] = df[text_col].astype(str).apply(lambda t: TextBlob(t).sentiment.polarity)
    df["_tb_subjectivity"] = df[text_col].astype(str).apply(lambda t: TextBlob(t).sentiment.subjectivity)
    df["_tb_label"] = df["_tb_polarity"].apply(lambda s: polarity_to_label(s))
    tb_exec_time = time.time() - t1

# PAGE: VADER Sentiment
if page == "VADER Sentiment":
    st.header("🔹 VADER Sentiment")
    st.markdown("VADER produces a **compound** polarity score in [-1,1]. We show numeric values and derived labels.")
    st.metric("Execution time (VADER compute)", f"{vader_exec_time:.4f} sec")
    st.write("### Sample (text, compound score, label)")
    st.dataframe(df[[text_col, "_vader_compound", "_vader_label"]].head(15))

    # Distribution
    st.write("### Label distribution (VADER)")
    vcounts = df["_vader_label"].value_counts().reindex(["Positive","Neutral","Negative"], fill_value=0)
    st.bar_chart(vcounts)

    st.write("#### Most positive (by compound)")
    st.dataframe(df[[text_col, "_vader_compound"]].sort_values("_vader_compound", ascending=False).head(5))
    st.write("#### Most negative (by compound)")
    st.dataframe(df[[text_col, "_vader_compound"]].sort_values("_vader_compound", ascending=True).head(5))

# PAGE: TextBlob Sentiment
elif page == "TextBlob Sentiment":
    st.header("📝 TextBlob Sentiment")
    st.markdown("TextBlob returns **polarity** [-1,1] and **subjectivity** [0,1]. We show numeric polarity, subjectivity and derived labels.")
    st.metric("Execution time (TextBlob compute)", f"{tb_exec_time:.4f} sec")
    st.write("### Sample (text, polarity, subjectivity, label)")
    st.dataframe(df[[text_col, "_tb_polarity", "_tb_subjectivity", "_tb_label"]].head(15))

    # Distribution
    st.write("### Label distribution (TextBlob)")
    tbcounts = df["_tb_label"].value_counts().reindex(["Positive","Neutral","Negative"], fill_value=0)
    st.bar_chart(tbcounts)

    st.write("#### Most positive (by polarity)")
    st.dataframe(df[[text_col, "_tb_polarity"]].sort_values("_tb_polarity", ascending=False).head(5))
    st.write("#### Most negative (by polarity)")
    st.dataframe(df[[text_col, "_tb_polarity"]].sort_values("_tb_polarity", ascending=True).head(5))

# PAGE: Comparison Dashboard
else:
    st.header("⚖️ Comparison Dashboard")
    st.markdown("Compare execution times, label counts, and average polarity.")

    col1, col2 = st.columns(2)
    col1.metric("VADER compute time", f"{vader_exec_time:.4f} sec")
    col2.metric("TextBlob compute time", f"{tb_exec_time:.4f} sec")

    comp = pd.DataFrame({
        "Method": ["VADER", "TextBlob"],
        "Positive": [int((df["_vader_label"]=="Positive").sum()), int((df["_tb_label"]=="Positive").sum())],
        "Neutral":  [int((df["_vader_label"]=="Neutral").sum()),  int((df["_tb_label"]=="Neutral").sum())],
        "Negative": [int((df["_vader_label"]=="Negative").sum()), int((df["_tb_label"]=="Negative").sum())],
        "Avg Polarity": [df["_vader_compound"].mean(), df["_tb_polarity"].mean()]
    })
    st.dataframe(comp)

    st.write("### Sentiment counts comparison")
    chart_df = comp.set_index("Method")[["Positive","Neutral","Negative"]].T
    st.bar_chart(chart_df)

    st.write("### Sample rows with both methods' labels")
    st.dataframe(df[[text_col, "_vader_compound","_vader_label","_tb_polarity","_tb_label"]].head(20))

# Download section
st.markdown("---")
download_cols = [text_col]
for c in ["_vader_compound","_vader_label","_tb_polarity","_tb_subjectivity","_tb_label"]:
    if c in df.columns:
        download_cols.append(c)
csv = df[download_cols].to_csv(index=False).encode("utf-8")
st.download_button("📥 Download processed results", data=csv, file_name="sentiment_processed.csv", mime="text/csv")


Overwriting app.py


In [ ]:
!ngrok config add-authtoken 34YNTAGVw3aYDYO61Rg8LIQjmf1_5N8ojgpMFqA3ayw73vEEE


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok
import threading, os, time

# Kill existing processes
ngrok.kill()

# Start Streamlit in background
def run_app():
    os.system("streamlit run app.py --server.port 8501")

thread = threading.Thread(target=run_app, daemon=True)
thread.start()

# Wait for Streamlit to boot up
time.sleep(8)

# Create public URL
public_url = ngrok.connect(8501)
print("✅ Streamlit App Running at:", public_url)


✅ Streamlit App Running at: NgrokTunnel: "https://undeviatingly-vacillant-sharyn.ngrok-free.dev" -> "http://localhost:8501"
